In [ ]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
import matplotlib.animation as mpl_animation
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle, Rectangle

In [ ]:
ped_path = '/home/flutz/PycharmProjects/social-force-model/evaluation/vci-dataset-citr-master/data/trajectories_filtered/vci_front/front_interaction_01_traj_ped_filtered.csv'
veh_path = '/home/flutz/PycharmProjects/social-force-model/evaluation/vci-dataset-citr-master/data/trajectories_filtered/vci_front/front_interaction_01_traj_veh_filtered.csv'

In [ ]:
ped_df = pd.read_csv(ped_path)
ped_df

In [ ]:
ped_df.rename(columns={'id': 'ped_id', 'x_est': 'x', 'y_est': 'y', 'vx_est': 'v_x', 'vy_est': 'v_y'}, inplace=True)
ped_df['frame'] = ped_df['frame'] - ped_df['frame'].min()
ped_df['y'] = ped_df['y']*-1
ped_df['v_y'] = ped_df['v_y']*-1
ped_df.sort_values(by=['ped_id', 'frame'], inplace=True)
ped_df

In [ ]:
if veh_path:
    veh_df = pd.read_csv(veh_path)
    veh_df.rename(columns={'id': 'veh_id', 'x_est': 'x', 'y_est': 'y', 'psi_est': 'heading', 'vel_est': 'vel'}, inplace=True)
    veh_df['frame'] = veh_df['frame'] - veh_df['frame'].min()
    veh_df['y'] = veh_df['y']*-1
    veh_df['heading'] = veh_df['heading']*-1
    veh_df.sort_values(by=['veh_id', 'frame'], inplace=True)
else:
    veh_df = None
veh_df

In [ ]:
fig, ax = plt.subplots()

for key, grp in ped_df.groupby(['ped_id']):
    ax = grp.plot(ax=ax, kind='line', x='x', y='y', label=f'ped_{key}')
    ax.arrow(grp['x'].iloc[0], grp['y'].iloc[0], grp['x'].iloc[10] - grp['x'].iloc[0], grp['y'].iloc[10] - grp['y'].iloc[0], width=0.1, zorder=10)

if veh_path:
    for key, grp in veh_df.groupby(['veh_id']):
        ax = grp.plot(ax=ax, kind='line', ls='--', lw=2, x='x', y='y', label=f'veh_{key}')
        ax.arrow(grp['x'].iloc[0], grp['y'].iloc[0], grp['x'].iloc[10] - grp['x'].iloc[0], grp['y'].iloc[10] - grp['y'].iloc[0], width=0.2, zorder=10)
        
ax.set_aspect('equal')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')    
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
def plot_human(step):
    """Generate patches for human
    :param step: index of state, default is the latest
    :return: list of patches
    """
    current_state = ped_df.loc[ped_df['frame'] == step]
    radius = [0.2] * current_state.shape[0]
   
    human_actors = [Circle((x, y), radius=r) for x, y, r in zip(current_state[['x']].to_numpy(), current_state[['y']].to_numpy(), radius)]
    human_collection.set_paths(human_actors)
    human_collection.set_array(np.arange(current_state.shape[0]))

    
def plot_vehicle(step):
    current_state = veh_df.loc[veh_df['frame'] == step]
    veh_length = 2.4
    veh_width = 1.2
    vehicle_actors = [Rectangle((x - veh_length/2, y - veh_width/2), width=veh_length, height=veh_width, angle=np.rad2deg(heading), rotation_point='center') 
                      for x, y, heading in zip(current_state[['x']].to_numpy(), current_state[['y']].to_numpy(), current_state[['heading']].to_numpy())]
    vehicle_collection.set_paths(vehicle_actors)
    vehicle_collection.set_array(np.arange(current_state.shape[0]))
    
        
def animation_init():
    ax.add_collection(human_collection)
    if veh_path:
        ax.add_collection(vehicle_collection)

    return human_collection, vehicle_collection

def animation_update(i):
    plot_human(i)
    if veh_path:
        plot_vehicle(i)
    return human_collection, vehicle_collection

In [ ]:
human_actors = None
human_collection = PatchCollection([])
human_collection.set(animated=True, alpha=0.6, cmap='viridis', clip_on=True)

vehicle_actors = None
vehicle_collection = PatchCollection([])
vehicle_collection.set(animated=True, alpha=0.6, cmap='viridis', clip_on=True)

fig, ax = plt.subplots()

 # x, y limit from states, only for animation
margin = 2.0
x_min = ped_df['x'].min() - margin
x_max = ped_df['x'].max() + margin
y_min = ped_df['y'].min() - margin
y_max = ped_df['y'].max() + margin

ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))
ax.set_aspect('equal')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')

frames = ped_df['frame'].max()

ani = mpl_animation.FuncAnimation(
            fig,
            init_func=animation_init,
            func=animation_update,
            frames=frames,
            interval=50)
ani